In [1]:

import psycopg2
from psycopg2 import sql
import requests
from datetime import datetime, timezone
import json
import re
import openai
import numpy

# endpoint = "https://active-monitor-48.hasura.app/v1/graphql"
# admin_key = "bAQuK7HSYvMAp6S6pnqXH0wQlyuKNUICzoW3jwecc27pwz6COLhE750s5YAec7Hz"

endpoint = "https://one-cicada-96.hasura.app/v1/graphql"
admin_key = "CQ3Je2OijkPamj0HVw7UBVNA56R9Z3fidRwVP5EsuIvPA2NsM3O0vcqx81F44JVl"

def query_hasura_graphql(endpoint, admin_key, query, variables):
    headers = {
        'Content-Type': 'application/json',
        'x-hasura-admin-secret': f'{admin_key}'
    }

    data = {
        'query': query,
        'variables': variables
    }
    response = requests.post(endpoint, json=data, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        # print(f"Request failed with status code {response.status_code}")
        return None

def mutation_hasura_graphql(endpoint, admin_key, mutation_query, mutation_variables):
    headers = {
        'Content-Type': 'application/json',
        'x-hasura-admin-secret': f'{admin_key}'
    }
    response = requests.post(endpoint, json={'query': mutation_query, 'variables': mutation_variables}, headers=headers)
    if response.ok:
        data = response.json()
        # print(data)
        return True, data
    else:
        # print(f"Mutation failed with status code {response.status_code}: {response.text}")
        return False, None

def create_conn_source(select_query):
    conn = None
    try:
        conn = psycopg2.connect(
            host="ep-purple-boat-77462220.ap-southeast-1.aws.neon.tech",
            database="neondb",
            user="gskchaitanya.gadde",
            password="aWO71xgmLjUv"
        )
        cur = conn.cursor()
        cur.execute(select_query)
        rows = cur.fetchall()
        cur.close()
        return rows
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

def update_conn_source(update_query):
    conn = None
    try:
        conn = psycopg2.connect(
            host="ep-purple-boat-77462220.ap-southeast-1.aws.neon.tech",
            database="neondb",
            user="gskchaitanya.gadde",
            password="aWO71xgmLjUv"
        )
        cur = conn.cursor()
        cur.execute(update_query)
        conn.commit()
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

def insert_conn_source(insert_query, data):
    conn = None

    try:
        conn = psycopg2.connect(
            host="ep-purple-boat-77462220.ap-southeast-1.aws.neon.tech",
            database="neondb",
            user="gskchaitanya.gadde",
            password="aWO71xgmLjUv"
        )
        cur = conn.cursor()
        for item in data:
            #print(item)
            cur.execute(insert_query, item)
        conn.commit()
        cur.close()
        # print("Data inserted successfully")
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

def select_conn_destination(select_query):
    conn = None
    try:
        conn = psycopg2.connect(
            host="aws-0-ap-south-1.pooler.supabase.com",
            database="postgres",
            user="postgres.icrispjgfllulboelvhw",
            password="rJywtQ5R2Wt17hG1",
            port=6543
        )
        cur = conn.cursor()
        cur.execute(select_query)
        rows = cur.fetchall()
        cur.close()
        return rows
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

def create_conn_destination(insert_query, data):
    conn = None

    try:
        conn = psycopg2.connect(
            host="aws-0-ap-south-1.pooler.supabase.com",
            database="postgres",
            user="postgres.icrispjgfllulboelvhw",
            password="rJywtQ5R2Wt17hG1",
            port=6543
        )
        cur = conn.cursor()
        for item in data:
            #print(item)
            cur.execute(insert_query, item)
        conn.commit()
        cur.close()
        # print("Data inserted successfully")
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

def update_conn_destination(update_query):
    conn = None

    try:
        conn = psycopg2.connect(
            host="aws-0-ap-south-1.pooler.supabase.com",
            database="postgres",
            user="postgres.icrispjgfllulboelvhw",
            password="rJywtQ5R2Wt17hG1",
            port=6543
        )
        cur = conn.cursor()
        cur.execute(update_query)
        conn.commit()
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            
def is_valid_timezone_format(published):
    try:
        # Attempt to parse the string
        date_format = "%a, %d %b %Y %H:%M:%S %z"
        date_object = datetime.strptime(published, date_format)

        hasura_timestamp = date_object.astimezone(timezone.utc).isoformat()
        return True, hasura_timestamp
    except ValueError:
        # If parsing fails, the string is not in the correct format
        return False, None

def check_date_format(date_string):
    try:
        datetime.strptime(date_string, '%Y-%m-%dT%H:%M:%S%z')
        return True
    except ValueError:
        return False        

def extract_json(s):
    start = s.find('{')
    end = s.rfind('}') + 1  # +1 to include the '}' in the substring
    json_str = s[start:end]
    try:
        return json.loads(json_str)
    except json.JSONDecodeError:
        return None

def suffix(d):
    return 'th' if 11<=d<=13 else {1:'st',2:'nd',3:'rd'}.get(d%10, 'th')

def get_part_of_day(hour):
    return (
        "Morning" if 5 <= hour <= 11
        else
        "Afternoon" if 12 <= hour <= 17
        else
        "Evening" if 18 <= hour <= 22
        else
        "Night"
    )



In [11]:
q1 =    """
query MyQuery {
  synopse_articles_t_v7_tag_play {
    id
    filename
  }
}
"""
v1 = {}	
m1 = """
mutation MyMutation($_eq: bigint = "", $filename: String = "") {
  update_synopse_articles_t_v7_tag_play(where: {id: {_eq: $_eq}}, _set: {filename: $filename}) {
    affected_rows
  }
}
"""
response_data = query_hasura_graphql(endpoint, admin_key, q1, v1)
print(response_data)
for i in response_data['data']['synopse_articles_t_v7_tag_play']:
    new_filename = i['filename'][:-3] + 'wav'
    v2 = {'_eq': i['id'], 'filename': new_filename}
    mutation_hasura_graphql(endpoint, admin_key, m1, v2)

{'data': {'synopse_articles_t_v7_tag_play': [{'id': 9, 'filename': '457_2024022608.mp3'}, {'id': 10, 'filename': '459_2024022608.mp3'}, {'id': 11, 'filename': '461_2024022608.mp3'}, {'id': 12, 'filename': '465_2024022608.mp3'}, {'id': 13, 'filename': '468_2024022608.mp3'}, {'id': 14, 'filename': '476_2024022608.mp3'}, {'id': 15, 'filename': '478_2024022608.mp3'}, {'id': 16, 'filename': '480_2024022608.mp3'}, {'id': 17, 'filename': '472_2024022608.mp3'}, {'id': 48, 'filename': '493_2024022608.mp3'}, {'id': 18, 'filename': '481_2024022608.mp3'}, {'id': 19, 'filename': '482_2024022608.mp3'}, {'id': 20, 'filename': '483_2024022608.mp3'}, {'id': 21, 'filename': '484_2024022608.mp3'}, {'id': 22, 'filename': '487_2024022608.mp3'}, {'id': 23, 'filename': '488_2024022608.mp3'}, {'id': 24, 'filename': '494_2024022608.mp3'}, {'id': 25, 'filename': '497_2024022608.mp3'}, {'id': 26, 'filename': '501_2024022608.mp3'}, {'id': 27, 'filename': '502_2024022608.mp3'}, {'id': 28, 'filename': '503_20240226

In [13]:

from dataplane import s3_upload
import boto3
from botocore.client import Config
# Define S3 credentials and bucket name

AccountID = "36fd753272b7c0c0c7a336fc277ebc15"
Bucket =  "synopse-play"
ClientAccessKey = "079d987e48e7681c73b67a12059b9bd4"
ClientSecret = "95d7fec11edf4d332ab1aeffba985e68c38b26aefbf0c6ac45dedab1e0da5817"
ConnectionUrl = f"https://{AccountID}.r2.cloudflarestorage.com"

S3Connect = boto3.client(
    's3',
    endpoint_url=ConnectionUrl,
    aws_access_key_id=ClientAccessKey,
    aws_secret_access_key=ClientSecret,
    config=Config(signature_version='s3v4'),
    region_name='us-east-1'
)
with open('output.wav', 'rb') as f:
    file_contents = f.read()
    
rs = s3_upload(
    Bucket=Bucket, 
    S3Client=S3Connect,
    TargetFilePath='output.wav',
    SourceFilePath='output.wav',
    UploadObject=file_contents,
    UploadMethod="Object"
)

In [ ]:
import os
from IPython.display import Audio
import nltk  # we'll use this to split into sentences
import numpy as np
from bark.generation import (
    generate_text_semantic,
    preload_models,
)
from bark.api import semantic_to_waveform
from bark import generate_audio, SAMPLE_RATE
import re
from mutagen.mp3 import MP3
import random
from pydub import AudioSegment
import torch
from GPUtil import showUtilization as gpu_usage
from numba import cuda
import soundfile as sf
from mutagen.mp3 import MP3
import psycopg2
from psycopg2 import sql
import requests
from dataplane import s3_upload
import boto3
from botocore.client import Config

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
preload_models()
nltk.download('punkt')

AccountID = "36fd753272b7c0c0c7a336fc277ebc15"
Bucket =  "synopse-play"
ClientAccessKey = "079d987e48e7681c73b67a12059b9bd4"
ClientSecret = "95d7fec11edf4d332ab1aeffba985e68c38b26aefbf0c6ac45dedab1e0da5817"
ConnectionUrl = f"https://{AccountID}.r2.cloudflarestorage.com"

S3Connect = boto3.client(
    's3',
    endpoint_url=ConnectionUrl,
    aws_access_key_id=ClientAccessKey,
    aws_secret_access_key=ClientSecret,
    config=Config(signature_version='s3v4'),
    region_name='us-east-1'
)

def select_conn_destination(select_query):
    conn = None
    try:
        conn = psycopg2.connect(
            host="aws-0-ap-south-1.pooler.supabase.com",
            database="postgres",
            user="postgres.icrispjgfllulboelvhw",
            password="rJywtQ5R2Wt17hG1",
            port=6543
        )
        cur = conn.cursor()
        cur.execute(select_query)
        rows = cur.fetchall()
        cur.close()
        return rows
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

def update_conn_destination(update_query):
    conn = None

    try:
        conn = psycopg2.connect(
            host="aws-0-ap-south-1.pooler.supabase.com",
            database="postgres",
            user="postgres.icrispjgfllulboelvhw",
            password="rJywtQ5R2Wt17hG1",
            port=6543
        )
        cur = conn.cursor()
        cur.execute(update_query)
        conn.commit()
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

def play_gen():
  latest_play = """
  select a.filename, a.summary, a.id from synopse_articles.t_v7_tag_play a where a.is_audio_created = 0;
  """
  latest_play_output = select_conn_destination(latest_play)
  if len(latest_play_output) > 0:
    for i in range(len(latest_play_output)):
      script = latest_play_output[i][1].replace("\n", " ").strip()
      sentences = nltk.sent_tokenize(script)
      random_number = random.randint(1, 9)
      SPEAKER = "v2/en_speaker_" + str(random_number)
      silence = np.zeros(int(0.25 * SAMPLE_RATE))  # quarter second of silence
      pieces = []
      for sentence in sentences:
          audio_array = generate_audio(sentence, history_prompt=SPEAKER)
          pieces += [audio_array, silence.copy()]
      Audio(np.concatenate(pieces), rate=SAMPLE_RATE)
      sf.write(latest_play_output[i][1], np.concatenate(pieces), SAMPLE_RATE)
      with open(latest_play_output[i][1], 'rb') as f:
          file_contents = f.read()
      rs = s3_upload(
          Bucket=Bucket, 
          S3Client=S3Connect,
          TargetFilePath=latest_play_output[i][1],
          SourceFilePath=latest_play_output[i][1],
          UploadObject=file_contents,
          UploadMethod="Object"
          )
      ids_t = latest_play_output[i][2]
      u1= f"""UPDATE synopse_articles.t_v7_tag_play
                  SET is_audio_created = 1
                  WHERE id = {ids_t};"""
      update_conn_destination(u1)

def trending_play():
  latest_play = """
  select a.filename, a.summary, a.id from synopse_articles.t_v7_all_article_summary_8hours a where a.is_audio_created = 0;
  """
  latest_play_output = select_conn_destination(latest_play)
  if len(latest_play_output) > 0:
    for i in range(len(latest_play_output)):
      script = latest_play_output[i][1].replace("\n", " ").strip()
      sentences = nltk.sent_tokenize(script)
      random_number = random.randint(1, 9)
      SPEAKER = "v2/en_speaker_" + str(random_number)
      silence = np.zeros(int(0.25 * SAMPLE_RATE))  # quarter second of silence
      pieces = []
      for sentence in sentences:
          audio_array = generate_audio(sentence, history_prompt=SPEAKER)
          pieces += [audio_array, silence.copy()]
      Audio(np.concatenate(pieces), rate=SAMPLE_RATE)
      sf.write(latest_play_output[i][1], np.concatenate(pieces), SAMPLE_RATE)
      with open(latest_play_output[i][1], 'rb') as f:
          file_contents = f.read()
      rs = s3_upload(
          Bucket=Bucket, 
          S3Client=S3Connect,
          TargetFilePath=latest_play_output[i][1],
          SourceFilePath=latest_play_output[i][1],
          UploadObject=file_contents,
          UploadMethod="Object"
          )
      ids_t = latest_play_output[i][2]
      u1= f"""UPDATE synopse_articles.t_v7_all_article_summary_8hours
                  SET is_audio_created = 1
                  WHERE id = {ids_t};"""
      update_conn_destination(u1)

trending_play()
play_gen()

In [27]:

esecret_ANYSCALE_API_KEY = "esecret_7eix5t1gpk7a9t356htd89jn2g"
client = openai.OpenAI(
    base_url = "https://api.endpoints.anyscale.com/v1",
    api_key = esecret_ANYSCALE_API_KEY
)
now_utc = datetime.utcnow()
formatted_date = now_utc.strftime('%Y%m%d%H')
formatted_date_out1 = now_utc.strftime('%B %d') + suffix(now_utc.day)
date_int = int(formatted_date)

latest_play = """
select count(a.*) from synopse_articles.t_v7_tag_play a where a.created_at > (now() - '10:00:00'::interval);
"""
latest_play_output = select_conn_destination(latest_play)
if latest_play_output[0][0] > 0:
  tags = """
  SELECT a.tag_id, a.tag from synopse_articles.t_v4_tags_hierarchy_tree a;
  """
  tags_output = select_conn_destination(tags)
  for i in range(len(tags_output)):
      article1 = """
      SELECT COALESCE(count(a.*), 0), b.article_group_id, b.title, b.summary
      FROM synopse_articles.t_v4_article_groups_l2_detail AS b 
      LEFT JOIN synopse_realtime.t_user_article_views AS a
      ON a.article_group_id = b.article_group_id
      where b.group_ai_tags_l3 = '""" + str(tags_output[i][1]) + """'
      and (a.created_at > (now() - '10:00:00'::interval) OR a.created_at IS NULL)
      group by b.article_group_id, b.title, b.summary
      order by COALESCE(count(a.*), 0) desc, b.article_group_id desc
      LIMIT 10;"""
      art1 = ""
      article1_output = select_conn_destination(article1)
      filename = str(tags_output[i][0]) + "_" + str(date_int) + ".mp3"
      for j in range(len(article1_output)):
          art1 += f"{article1_output[j][2]} - {article1_output[j][3]}\n"
      system_prompt = """
      you are news anchor reporting related to news category """ + str(tags_output[i][1]) + """ and you have to provide a transcript for a news anchor with around 75 words.
        Capture around 3 to 5 key points acknowledging diverse perspectives showcasing factual information without personal opinion.
        The keypoints should be informative and engaging, while the keyword points should be relevant to the content of the articles.
        Remember to strive for neutrality while remaining engaging and informative to focus on factual information by avoiding emotional language or sensationalism.
        Employ active voice and concise sentences for maximum clarity to ensure the summary is easily understood.
        provide and intro to the script and here are some sample intro make sure to follow a similar formar for intro
        
  "Good day and welcome to todays's edition of """ + str(tags_output[i][1])  +""" News, bringing you the latest updates from across the globe on """ + str(formatted_date_out1) + """ ."
  "Hello and thank you for joining us for  """ + str(tags_output[i][1])  +"""  News, where we dive into the advancements and innovations shaping our digital world, on """ + str(formatted_date_out1) + """ ."
  "Welcome to Synopse, your source for the most recent developments in  """ + str(tags_output[i][1])  +""" , broadcasting on """ + str(formatted_date_out1) + """ ."
  "Greetings, and thank you for tuning in to Synopse, where we unravel the latest news on """ + str(tags_output[i][1])  +""" , today on """ + str(formatted_date_out1) + """ ."
  "Good day, and welcome to Synopse, your go-to for the day's top news in  """ + str(tags_output[i][1])  +""" , broadcasting on """ + str(formatted_date_out1) + """ ."
        the entire script should be maximum of 150 words.
          """
      user_prompt = "all article data: " +  art1
      chat_completion = client.chat.completions.create(
        model="mistralai/Mixtral-8x7B-Instruct-v0.1",
        messages=[{"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}],
        max_tokens = 400,
        temperature = 0.3,
        top_p = 0.9
      )
      # print(chat_completion.choices[0].message.content)
      v1 = []
      v2 = []
      v2.append(tags_output[i][1])
      v2.append(tags_output[i][0])
      v2.append(chat_completion.choices[0].message.content)
      v2.append(filename)
      v1.append(tuple(v2))
      v1_insert_query ="INSERT INTO synopse_articles.t_v7_tag_play (tag_name, tag_id, summary, filename) VALUES (%s, %s, %s, %s)"
      create_conn_destination(v1_insert_query, v1)
    

In [4]:
latest_8hours = """
select count(a.*) from synopse_articles.t_v7_all_article_summary_8hours a where a.created_at > (now() - '10:00:00'::interval);
"""
latest_8hours_output = select_conn_destination(latest_8hours)

if latest_8hours_output[0][0] > 0:
    esecret_ANYSCALE_API_KEY = "esecret_7eix5t1gpk7a9t356htd89jn2g"
    client = openai.OpenAI(
        base_url = "https://api.endpoints.anyscale.com/v1",
        api_key = esecret_ANYSCALE_API_KEY
    )
    now_utc = datetime.utcnow()
    formatted_date = now_utc.strftime('%Y%m%d%H')
    formatted_date_out1 = now_utc.strftime('%B %d') + suffix(now_utc.day)  + ', ' + get_part_of_day(now_utc.hour)
    date_int = int(formatted_date)
    article1 = """
    SELECT COALESCE(count(a.*), 0), b.article_group_id, b.title, b.summary
    FROM synopse_articles.t_v4_article_groups_l2_detail AS b 
    JOIN synopse_realtime.t_user_article_views AS a
    ON a.article_group_id = b.article_group_id
    and (a.created_at > (now() - '15:00:00'::interval) OR a.created_at IS NULL)
    group by b.article_group_id, b.title, b.summary
    order by COALESCE(count(a.*), 0) desc, b.article_group_id desc
    LIMIT 10;"""
    art1 = ""
    article1_output = select_conn_destination(article1)
    title = ""
    for j in range(len(article1_output)):
        title += f"{article1_output[j][2]}\n"
        art1 += f"{article1_output[j][2]} - {article1_output[j][3]}\n"
    filename = "trending_" + str(date_int) + ".mp3"
    system_prompt = """
    you are news anchor reporting trending news and you have to provide a transcript for a news anchor with around 150 words.
    Capture around 5 to 7 key points acknowledging diverse perspectives showcasing factual information without personal opinion.
    The keypoints should be informative and engaging, while the keyword points should be relevant to the content of the articles.
    Remember to strive for neutrality while remaining engaging and informative to focus on factual information by avoiding emotional language or sensationalism.
    Employ active voice and concise sentences for maximum clarity to ensure the summary is easily understood.
    provide and intro to the script and here are some sample intro lines modify and provide a similar intro for the script

    "Good evening and welcome to tonight's edition of trending news. It's  """ + str(formatted_date_out1) + """ , and we've got the latest updates on stories making waves around the globe."
    "Hello, and thank you for joining us for trending news on this  """ + str(formatted_date_out1) + """ . From breaking developments to viral sensations, we've got you covered."
    "From the heart of the newsroom, this is your update on trending headlines for  """ + str(formatted_date_out1) + """ . Sit tight as we delve into the stories captivating the world's attention."
    "Welcome to our broadcast on this  """ + str(formatted_date_out1) + """ , where we bring you the hottest topics dominating social media feeds and news cycles alike."
    "It's  """ + str(formatted_date_out1) + """ , and you're tuned in to our trending news segment. Stay with us as we explore the most talked-about stories of the day."
    "Good day, everyone, and thank you for joining us for trending news. I'm here to bring you the latest buzz and updates from around the world as of  """ + str(formatted_date_out1) + """ ."
    "Greetings, viewers. As we kick off  """ + str(formatted_date_out1) + """ , let's dive into the trending news stories making headlines and sparking conversations globally."
    "Welcome aboard our trending news express. Buckle up as we navigate through the top stories dominating the internet and airwaves on this  """ + str(formatted_date_out1) + """ ."
    "It's  """ + str(formatted_date_out1) + """ , and you're watching our trending news roundup. Stick around as we unpack the day's most significant events and trending topics."
    "Hello, and a warm welcome to our trending news segment. On this  """ + str(formatted_date_out1) + """ , we're here to bring you up to speed on the stories everyone's talking about."
    the entire script should be maximum of 350 words.
    """
    user_prompt = "all article data: " +  art1
    chat_completion = client.chat.completions.create(
    model="mistralai/Mixtral-8x7B-Instruct-v0.1",
    messages=[{"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}],
    max_tokens = 1000,
    temperature = 0.3,
    top_p = 0.9
    )
    # print(chat_completion.choices[0].message.content)
    v1 = []
    v2 = []
    v2.append("Trending News: " + str(formatted_date_out1) + " : " + title)
    v2.append(chat_completion.choices[0].message.content)
    v2.append(filename)
    v1.append(tuple(v2))
    v1_insert_query ="INSERT INTO synopse_articles.t_v7_all_article_summary_8hours (title, summary, filename) VALUES (%s, %s, %s)"
    create_conn_destination(v1_insert_query, v1)
        

In [6]:
latest_8hours = """
select count(a.*) from synopse_articles.t_v7_all_article_summary_8hours a where a.created_at > (now() - '10:00:00'::interval);
"""
latest_8hours_output = select_conn_destination(latest_8hours)
print(latest_8hours_output[0][0] )

1


In [3]:

translations = {
    "Technology, Business & Economy": "టెక్నాలజీ, వ్యాపారం & ఆర్థిక విభాగం",
    "Film Industry": "సినిమా పరిశ్రమ",
    "Society & Lifestyle": "సమాజం & జీవన శైలి",
    "Food & Cuisine": "ఆహారం & పాకం",
    "Political & Elections": "రాజకీయ మరియు ఎన్నికలు",
    "Well-being": "ఆరోగ్య సంబంధితమైన",
    "Sports & Recreation": "క్రీడలు & విశ్రాంతి",
    "Gaming": "ఆటలు",
    "Physics": "భౌతిక శాస్త్రం",
    "Astronomy": "జ్యోతిష్యం",
    "Geopolitics": "జియోపాలిటిక్స్",
    "Government & Policies": "ప్రభుత్వం & నయాలు",
    "Terrorism": "ఆతంకాలు",
    "Security": "భద్రత",
    "Financial": "ఆర్థిక",
    "Stocks & Investing": "స్టాక్స్ & నివేశం",
    "Cricket": "క్రికెట్",
    "Tennis": "టెన్నిస్",
    "Golf": "గోల్ఫ్",
    "Motorsports": "మోటర్‌స్పోర్ట్స్",
    "Athletics": "ఆథ్‌లెటిక్స్",
    "Internet Of Things": "వస్తువుల ఇంటర్నెట్",
    "Computing": "కంప్యూటింగ్",
    "Climate": "జలవాయు",
    "Culture & Entertainment": "సంస్కృతి & వినోదం",
    "Politics & Global Affairs": "రాజకీయ మరియు గ్లోబల్ విషయాలు",
    "Education & Learning": "విద్య & అభ్యాసం",
    "Football (Soccer)": "ఫుట్‌బాల్ (సాక్షరత)",
    "Combat Sports": "యుద్ధ క్రీడలు",
    "Basketball": "బాస్కెట్‌బాల్",
    "Mythology": "పురాణాలు",
    "Baseball": "బేస్‌బాల్",
    "Aerospace": "ఎయిరోస్పేస్",
    "Pollution": "కాలుష్యం",
    "Artificial Intelligence": "కృత్రిమ మేధస్సు",
    "Higher Education": "ప్రభుత్వ ఉన్నత విద్య",
    "Environment & Health": "పరిసరం & ఆరోగ్యం",
    "Mental Health": "మానసిక ఆరోగ్యం",
    "Linguistics": "భాషా శాస్త్రం",
    "Crime": "అపరాధం",
    "Social Issues": "సామాజిక సమస్యలు",
    "Robotics": "రోబోటిక్స్",
    "Wearables": "ధరించబడే",
    "Biotechnology": "బెయోటెక్నాలజీ",
    "Manufacturing": "నిర్మాణం",
    "Software": "సాఫ్ట్‌వేర్",
    "Travel": "ప్రయాణం",
    "Design": "రచన",
    "Home Products": "హోమ్ ఉత్పత్తులు",
    "Swimming": "సంవారించడం",
    "E-sports": "ఇ-క్రీడలు",
    "Outdoor Recreation": "బాహ్య విశ్రాంతి",
    "Hockey": "హాకీ",
    "sponsorships": "స్పాన్సర్షిప్స్",
    "American football": "అమెరికన్ ఫుట్‌బాల్",
    "Badminton": "బ్యాడ్‌మింటన్",
    "Schooling": "పాఠశాలలు",
    "Flora and Fauna": "ప్రాణువులు మరియు పూలు",
    "Natural Disasters": "ప్రాకృతిక ఆపదలు",
    "Healthcare": "ఆరోగ్య సేవలు",
    "Automobile": "ఆటోమొబైల్",
    "Telecommunications": "టెలికామ్యూనికేషన్లు",
    "Energy": "ఎనర్జీ"
}

# Input data
input_data = {
    "data": {
        "synopse_articles_t_v4_tags_hierarchy": [
            {"tag": "Technology, Business & Economy"},
            {"tag": "Film Industry"},
            {"tag": "Society & Lifestyle"},
            {"tag": "Food & Cuisine"},
            {"tag": "Political & Elections"},
            {"tag": "Well-being"},
            {"tag": "Sports & Recreation"},
            {"tag": "Gaming"},
            {"tag": "Physics"},
            {"tag": "Astronomy"},
            {"tag": "Geopolitics"},
            {"tag": "Government & Policies"},
            {"tag": "Terrorism"},
            {"tag": "Security"},
            {"tag": "Financial"},
            {"tag": "Stocks & Investing"},
            {"tag": "Cricket"},
            {"tag": "Tennis"},
            {"tag": "Golf"},
            {"tag": "Motorsports"},
            {"tag": "Athletics"},
            {"tag": "Internet Of Things"},
            {"tag": "Computing"},
            {"tag": "Climate"},
            {"tag": "Culture & Entertainment"},
            {"tag": "Politics & Global Affairs"},
            {"tag": "Education & Learning"},
            {"tag": "Football (Soccer)"},
            {"tag": "Combat Sports"},
            {"tag": "Basketball"},
            {"tag": "Mythology"},
            {"tag": "Baseball"},
            {"tag": "Aerospace"},
            {"tag": "Pollution"},
            {"tag": "Artificial Intelligence"},
            {"tag": "Higher Education"},
            {"tag": "Environment & Health"},
            {"tag": "Mental Health"},
            {"tag": "Linguistics"},
            {"tag": "Crime"},
            {"tag": "Social Issues"},
            {"tag": "Robotics"},
            {"tag": "Wearables"},
            {"tag": "Biotechnology"},
            {"tag": "Manufacturing"},
            {"tag": "Software"},
            {"tag": "Travel"},
            {"tag": "Design"},
            {"tag": "Home Products"},
            {"tag": "Swimming"},
            {"tag": "E-sports"},
            {"tag": "Outdoor Recreation"},
            {"tag": "Hockey"},
            {"tag": "sponsorships"},
            {"tag": "American football"},
            {"tag": "Badminton"},
            {"tag": "Schooling"},
            {"tag": "Flora and Fauna"},
            {"tag": "Natural Disasters"},
            {"tag": "Healthcare"},
            {"tag": "Automobile"},
            {"tag": "Telecommunications"},
            {"tag": "Energy"}
        ]
    }
}

# Process input data to generate output
output_data = []
for item in input_data["data"]["synopse_articles_t_v4_tags_hierarchy"]:
    tag = item["tag"]
    if tag in translations:
        translated_tag = translations[tag]
        output_data.append({"tag": tag, "tag_tel": translated_tag})

# Output the result in JSON format
import json
output_json = json.dumps(output_data, ensure_ascii=False, indent=2)
print(output_json)

[
  {
    "tag": "Technology, Business & Economy",
    "tag_tel": "టెక్నాలజీ, వ్యాపారం & ఆర్థిక విభాగం"
  },
  {
    "tag": "Film Industry",
    "tag_tel": "సినిమా పరిశ్రమ"
  },
  {
    "tag": "Society & Lifestyle",
    "tag_tel": "సమాజం & జీవన శైలి"
  },
  {
    "tag": "Food & Cuisine",
    "tag_tel": "ఆహారం & పాకం"
  },
  {
    "tag": "Political & Elections",
    "tag_tel": "రాజకీయ మరియు ఎన్నికలు"
  },
  {
    "tag": "Well-being",
    "tag_tel": "ఆరోగ్య సంబంధితమైన"
  },
  {
    "tag": "Sports & Recreation",
    "tag_tel": "క్రీడలు & విశ్రాంతి"
  },
  {
    "tag": "Gaming",
    "tag_tel": "ఆటలు"
  },
  {
    "tag": "Physics",
    "tag_tel": "భౌతిక శాస్త్రం"
  },
  {
    "tag": "Astronomy",
    "tag_tel": "జ్యోతిష్యం"
  },
  {
    "tag": "Geopolitics",
    "tag_tel": "జియోపాలిటిక్స్"
  },
  {
    "tag": "Government & Policies",
    "tag_tel": "ప్రభుత్వం & నయాలు"
  },
  {
    "tag": "Terrorism",
    "tag_tel": "ఆతంకాలు"
  },
  {
    "tag": "Security",
    "tag_tel": "భద్రత"
  },
  {
   

In [3]:
argss = []
argsss = ['andhrajyothyi', 'eenadu', 'oneindia', 'faz', 'tagesschau', 'timesofindia', 'thehindu', 'cnn', 'foxnews', 'cbsnews', 'espn', 'forbes', 'abcnews', 'huffpost', 'nytimes', 'reuters', 'nbc', 'wsj', 'politico', 'independent','techcrunch', 'the verge', 'vox', 'thewashingtonpost', 'aljazeera', 'wired', 'marketwatch', 'newyorkpost', 'thewire', 'appleinsider','wion', 'theindianexpress', 'hindustantimes', 'latestly', 'thedailybeast', 'tv9telugu', 'ntvtelugu', 'teluguht', 'republicbharat', 'dainikbhaskar','amarujala', 'livehindustan', 'rajasthanchronicle', 'punjabkesari', 'prabhatkhabar', 'naidunia', 'haribhoomi', 'moneycontrol', 'oneindiahindi', 'zeenewshindi','news24onlinehindi', 'patrika', 'dailythanthi', 'maalaimalar', 'dinamalar', 'hindutamil', 'dinakaran', 'theekkathir', 'oneindiatamil', 'bz-berlin', 'zeit', 'swp-berlin', 'noz', 'Neue Presse', 'rnd', 'wp', 'derwesten', 'spiegel', 'focus']

for i in range(0, len(argsss), 10):
    argss.append(argsss[i:i+10])
print(argss)

[['andhrajyothyi', 'eenadu', 'oneindia', 'faz', 'tagesschau', 'timesofindia', 'thehindu', 'cnn', 'foxnews', 'cbsnews'], ['espn', 'forbes', 'abcnews', 'huffpost', 'nytimes', 'reuters', 'nbc', 'wsj', 'politico', 'independent'], ['techcrunch', 'the verge', 'vox', 'thewashingtonpost', 'aljazeera', 'wired', 'marketwatch', 'newyorkpost', 'thewire', 'appleinsider'], ['wion', 'theindianexpress', 'hindustantimes', 'latestly', 'thedailybeast', 'tv9telugu', 'ntvtelugu', 'teluguht', 'republicbharat', 'dainikbhaskar'], ['amarujala', 'livehindustan', 'rajasthanchronicle', 'punjabkesari', 'prabhatkhabar', 'naidunia', 'haribhoomi', 'moneycontrol', 'oneindiahindi', 'zeenewshindi'], ['news24onlinehindi', 'patrika', 'dailythanthi', 'maalaimalar', 'dinamalar', 'hindutamil', 'dinakaran', 'theekkathir', 'oneindiatamil', 'bz-berlin'], ['zeit', 'swp-berlin', 'noz', 'Neue Presse', 'rnd', 'wp', 'derwesten', 'spiegel', 'focus']]
